In [8]:
#导入所需数据模块
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer

In [4]:
#读取数据
df=pd.read_csv('Ch3-HousePrice-train.csv')

In [12]:
#计算缺失值分布情况
total=df.isnull().sum().sort_values(ascending=False)
percent=(df.isnull().sum()/df.isnull().count()).sort_values(ascending=False)
missing_data=pd.concat([total,percent],axis=1,keys=['Total','Percent'])
missing_data.head(20)

Total   Percent
PoolQC         1453  0.995205
MiscFeature    1406  0.963014
Alley          1369  0.937671
Fence          1179  0.807534
MasVnrType      872  0.597260
FireplaceQu     690  0.472603
LotFrontage     259  0.177397
GarageQual       81  0.055479
GarageFinish     81  0.055479
GarageType       81  0.055479
GarageYrBlt      81  0.055479
GarageCond       81  0.055479
BsmtFinType2     38  0.026027
BsmtExposure     38  0.026027
BsmtCond         37  0.025342
BsmtQual         37  0.025342
BsmtFinType1     37  0.025342
MasVnrArea        8  0.005479
Electrical        1  0.000685
Condition2        0  0.000000

观察数据缺失值可知，很多数据列空缺值极大，如果全部删除，将造成大量有效数据丢失，故在此要根据不同数据的意义和特点，制定不同缺失值处理方案。
本题数据关联性和空缺值填充方法参考了csdn博文https://blog.csdn.net/sinat_33519513/article/details/78965833  
在此感谢原作者！

In [19]:
#缺失量比较多的PoolQC、MiscFeature、Alley、Fence、FireplaceQu是由于房子没有泳池、特殊的设施、旁边的小巷、篱笆、壁炉等设施。
#我们可以选择直接剔除这些大量缺失的数据
#为了保证数据完整性，我们也可以直接用NA来填充

#这里选取第二种方法，用NA填充
df = df.fillna({
    'Alley' : 'NA',
    'FireplaceQu': 'NA',
    'Fence' : 'NA',
    'MiscFeature' : 'NAe',
    'MSSubClass':'NA',
    'PoolQC':'NA'
})

#如果选取第一种方法
#代码如下
#df=df.drop(missing_data[missing_data['Percent']>.4].index,axis=1)

In [16]:
#LotArea表示地块的大小，LotFrontage‌表示地块前部与街道的直线距离
#二者间有对应关系，利用多项式，基于area对frontage进行填充
x = df.loc[np.logical_not(df["LotFrontage"].isnull()), "LotArea"]
y = df.loc[np.logical_not(df["LotFrontage"].isnull()), "LotFrontage"]
t = (x <= 25000) & (y <= 150)
#拟合多项式以填充缺失值
p = np.polyfit(x[t], y[t], 1)
#根据多项式求函数值
df.loc[df['LotFrontage'].isnull(), 'LotFrontage'] = np.polyval(p, df.loc[df['LotFrontage'].isnull(), 'LotArea'])

In [24]:
#泳池也有第二种处理方法：考虑泳池面积与泳池质量的关系
#对于有泳池面积而没有泳池质量的利用对应的泳池面积进行填充
poolqcna = df[df['PoolQC'].isnull()& df['PoolArea']!=0][['PoolQC','PoolArea']]
areamean = df.groupby('PoolQC')['PoolArea'].mean()
for i in poolqcna.index:
    v = adf.loc[i,['PoolArea']].values
    df.loc[i,['PoolQC']] = np.abs(v-areamean).astype('float64').argmin()#此处必须加上.astype，否则出错，但3.6版本不会出错
#其他的直接填充没有泳池，面积填充0    
df['PoolQC'] = df["PoolQC"].fillna("None")
df['PoolArea'] =df["PoolArea"].fillna(0)

In [33]:
#对于MasVnr（外墙装饰），与泳池第二种方法类似
#有些房子重装修过，但是有些房子是没重装修过的。使用NA填充装修情况，用0填充装修面积。
MasVnrM = df.groupby('MasVnrType')['MasVnrArea'].median()
mtypena = df[(df['MasVnrType'].isnull())& (df['MasVnrArea'].notnull())][['MasVnrType','MasVnrArea']]
for i in mtypena.index:
    v = df.loc[i,['MasVnrArea']].values
    df.loc[i,['MasVnrType']] = np.abs(v-MasVnrM).astype('float64').argmin()

df['MasVnrType'] = df["MasVnrType"].fillna("NA")
df['MasVnrArea'] = df["MasVnrArea"].fillna(0)

In [36]:
#剩余缺失较少的属性，可以使用众数填充，保留原数据集的离散特征，不受极值影响
df["MSZoning"] = df.groupby("MSSubClass")["MSZoning"].transform(lambda x: x.fillna(x.mode()[0]))
df['Electrical']=df['Electrical'].fillna(df['Electrical'].mode()[0])
df['KitchenQual']=df['KitchenQual'].fillna(df['KitchenQual'].mode()[0])
df['Exterior1st']=df['Exterior1st'].fillna(df['Exterior1st'].mode()[0])
df['Exterior2nd']=df['Exterior2nd'].fillna(df['Exterior2nd'].mode()[0])
df['SaleType']=df['SaleType'].fillna(df['SaleType'].mode()[0])
df['Utilities']=df['Utilities'].fillna(df['Utilities'].mode()[0])
df['BsmtExposure']=df['BsmtExposure'].fillna(df['BsmtExposure'].mode()[0])
df['BsmtFinType2']=df['BsmtFinType2'].fillna(df['BsmtFinType2'].mode()[0])
df['BsmtCond']=df['BsmtCond'].fillna(df['BsmtCond'].mode()[0])
df['BsmtFinType1']=df['BsmtFinType1'].fillna(df['BsmtFinType1'].mode()[0])
df['BsmtQual']=df['BsmtQual'].fillna(df['BsmtQual'].mode()[0])
# Functional : data description says NA means typical
df["Functional"] = df["Functional"].fillna("Typ")

In [34]:
#garage车库,没有车库会造成车库相关属性缺失，对于这种情况，以NA填充
#定义garage相关的数组
garage_obj=['GarageType','GarageFinish','GarageQual','GarageCond','GarageYrBlt']
for gara in garage_obj:
    #df[gara].fillna("NA",inplace=True)
    df[gara] = df[gara].fillna("NA")

In [39]:
#而后重新计算缺失值分布情况
total=df.isnull().sum().sort_values(ascending=False)
percent=(df.isnull().sum()/df.isnull().count()).sort_values(ascending=False)
missing_data=pd.concat([total,percent],axis=1,keys=['Total','Percent'])
missing_data.head(20)
#运行后可知缺失值已经全部填充

Total  Percent
Id                0      0.0
MSSubClass        0      0.0
MSZoning          0      0.0
LotFrontage       0      0.0
LotArea           0      0.0
Street            0      0.0
Alley             0      0.0
LotShape          0      0.0
LandContour       0      0.0
Utilities         0      0.0
LotConfig         0      0.0
LandSlope         0      0.0
Neighborhood      0      0.0
Condition1        0      0.0
Condition2        0      0.0
BldgType          0      0.0
HouseStyle        0      0.0
OverallQual       0      0.0
OverallCond       0      0.0
YearBuilt         0      0.0